In [380]:
import pandas as pd
from datetime import datetime
from pathlib import Path 
import psycopg2 as sql
from sqlalchemy import create_engine

# Coleta de dados do site do IBGE

In [381]:
#Coleta dos dados
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

dadosIBGE = pd.DataFrame(pd.read_html(url, match= "Municípios do Rio de Janeiro")[0])
dadosIBGE

,Municípios do Rio de Janeiro,Códigos
0,Angra dos Reis,3300100
1,Aperibé,3300159
2,Araruama,3300209
3,Areal,3300225
4,Armação dos Búzios,3300233
...,...,...
87,Três Rios,3306008
88,Valença,3306107
89,Varre-Sai,3306156
90,Vassouras,3306206


In [382]:
# Renomeando colunas
dadosIBGE = dadosIBGE.rename(columns= {'Municípios do Rio de Janeiro': 'nmMunic', 'Códigos':'codMunic'})

#Alterando nome do index
dadosIBGE.index.name = 'idMunic'

#Aletrando para iniciar do 1
dadosIBGE.index = dadosIBGE.index +1

#Adicionando uma coluna de data e hora de carga

dtCarga = datetime.today().strftime('%m/%d/%y %H:%M')
dadosIBGE['dtCarga'] = dtCarga
dadosIBGE

,nmMunic,codMunic,dtCarga
idMunic,,,
1,Angra dos Reis,3300100,02/01/24 18:03
2,Aperibé,3300159,02/01/24 18:03
3,Araruama,3300209,02/01/24 18:03
4,Areal,3300225,02/01/24 18:03
5,Armação dos Búzios,3300233,02/01/24 18:03
...,...,...,...
88,Três Rios,3306008,02/01/24 18:03
89,Valença,3306107,02/01/24 18:03
90,Varre-Sai,3306156,02/01/24 18:03


In [383]:
dadosIBGE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 1 to 92
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   nmMunic   92 non-null     object
 1   codMunic  92 non-null     int64 
 2   dtCarga   92 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.3+ KB


# Conectando no banco de dados ODS

In [384]:
host = "localhost"  
port = "5432"       
database = "pascoaODS"  
user = "postgres"  
password = "*****"  

# Conectando ao PostgreSQL
try:
    connection = sql.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Conexão ao PostgreSQL bem-sucedida!")
except sql.Error as e:
    print("Erro ao conectar ao PostgreSQL:", e)


Conexão ao PostgreSQL bem-sucedida!


# MANIPULANDO OS BANCOS DE DADOS CRIADOS

In [385]:
# Carregar os dados do DataFrame dadosIBGE na tabela tbLogMunic

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")
dadosIBGE.to_sql('tbLogMunic', engine, if_exists="append", index=False)

# Importando dados de cadastro das delegacias de polícia

In [386]:
tbLogDP = pd.DataFrame(pd.read_csv('DP.CSV'))

#Criando coluna de data de carga
tbLogDP['dtCarga'] = datetime.today().strftime('%m/%d/%y %H:%M')
tbLogDP

,COD_DP,NM_DP,Endereço,dtCarga
0,1,001ª DP - Praça Mauá,Atendendo provisoriamente na sede da 4ª DP - P...,02/01/24 18:03
1,4,004ª DP - Praça da República,"Av. Presidente Vargas, 1100 - Centro, Rio de J...",02/01/24 18:03
2,5,005ª DP - Mem de Sá,"Avenida Gomes Freire, 320 - Centro, Rio de Jan...",02/01/24 18:03
3,6,006ª DP - Cidade Nova,"Rua Professor Clementino Fraga, 77 - Centro, R...",02/01/24 18:03
4,7,007ª DP - Santa Teresa,"Rua Francisco de Castro, 5 - Santa Teresa, Rio...",02/01/24 18:03
...,...,...,...,...
132,159,159ª DP - Cachoeira de Macacu,"Avenida Lord Baden Powel, 93 - Centro, Cachoei...",02/01/24 18:03
133,165,165ª DP - Mangaratiba,"Estrada São João Marcos, s/n - Praia do Saco, ...",02/01/24 18:03
134,166,166ª DP - Angra dos Reis,"Rua Doutor Coutinho, 6 - Centro, Angra dos Rei...",02/01/24 18:03
135,167,167ª DP - Paraty,"Rua Roberto da Silveira , s/n - Vila Colonial,...",02/01/24 18:03


# Importando dados dados de cadastro dos responsáveis pelas delegacias de polícia

In [387]:
tbLogRespDP = pd.DataFrame(pd.read_csv('ResponsavelDP.CSV'))

#Criando coluna de data de carga
tbLogRespDP['dtCarga'] = datetime.today().strftime('%m/%d/%y %H:%M')
tbLogRespDP

,COD_DP,Responsavel,dtCarga
0,1,Delegado de Polícia José Luiz Silva Duarte,02/01/24 18:03
1,4,Delegada de Polícia Patricia de Paiva Aguiar,02/01/24 18:03
2,5,Delegado de Polícia Bruno Gilaberte Freitas,02/01/24 18:03
3,6,Delegado de Polícia Fabio Luiz Da Silva Souza,02/01/24 18:03
4,7,Delegado de Polícia Carlos Alberto Meirelles D...,02/01/24 18:03
...,...,...,...
132,159,Delegado de Polícia Clovis Souza Moreira,02/01/24 18:03
133,165,Delegado de Polícia Anderson Ribeiro Pinto,02/01/24 18:03
134,166,Delegado de Polícia Vilson de Almeida Silva,02/01/24 18:03
135,167,Delegado de Polícia Marcelo dos Santos Haddad,02/01/24 18:03


# Conectando no ODS e informando ao python que usarei linguagem SQL

In [388]:
#Definição de manipulação de dados em SQL
SQL_ODS = connection.cursor()

# Inserindo registros na tabela tbLogDP

In [389]:
#Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogDP.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogDP (cod_DP,nm_DP,endereco,dtCarga) VALUES (%s, %s, %s, %s)''',tbLogDP.values.tolist())

#confirmar a transação
connection.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela tbLogDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# Inserindo registros na tabela tbLogRespDP

In [390]:
#Inserindo registros na tabela tbLogRespDP
print("Iniciando a inserção de dados na tabela tbLogRespDP.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogRespDP (cod_DP, Responsavel, dtCarga) VALUES (%s, %s, %s)''',tbLogRespDP.values.tolist())

#confirmar a transação
connection.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogRespDP), "registros inseridos na tbLogRespDP!")

Iniciando a inserção de dados na tabela tbLogRespDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogRespDP!


# Criando o dataframe com o select de criação da dimensão dp (ddp), no dw

In [391]:
qry_dDP = '''
SELECT cod_dp, nm_dp, endereco, responsavel
from
( SELECT
            a.cod_DP,
            a.nm_DP,
            a.endereco,
            b.responsavel,
            max(a.dtCarga)
        FROM tbLogDP a
        JOIN tbLogRespDP b
        on a.cod_DP = b.cod_DP
        GROUP BY
            a.cod_DP,
            a.nm_DP,
            a.endereco,
            b.Responsavel
	
	)a;
'''
dfdDP = pd.read_sql(qry_dDP, connection)

# Coletando dados dos batalhões de polícia militar (bpm)

In [392]:
#Criar Data Frame com os dados dos BPMs
tbLogBPM = pd.DataFrame(pd.read_csv("BPM.csv"))

#Criando coluna com a data da carga
tbLogBPM['dtCarga'] = dtCarga

# Construindo a tabela de log de registros do bpm (tbLogBPM), no ODS

In [393]:
#Definir a query de criação da tabela
qry_tbLogBPM = '''
    
    CREATE TABLE IF NOT EXISTS tbLogBPM (
    
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    dtCarga timestamp without time zone

    )
'''

#executar a criação da tbLogBPM
SQL_ODS.execute(qry_tbLogBPM)

#definição da qry de criação de index
qry_idx_tbLogBPM_codBPM = "CREATE INDEX IF NOT EXISTS idx_tbLogBPM_codBPM ON tbLogBPM (codBPM)"

#executar a criação do index
SQL_ODS.execute(qry_idx_tbLogBPM_codBPM)

# Inserindo dados na tbLogBPM

In [394]:
#Inserindo registros na tabela tbLogBPM
print("Iniciando a inserção de dados na tabela tbLogBPM.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogBPM (codBPM,nmBPM,enderecoBPM,dtCarga) VALUES (%s,%s,%s,%s)''',tbLogBPM.values.tolist())

#confirmar a transação
connection.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogBPM), "registros inseridos na tbLogBPM!")

Iniciando a inserção de dados na tabela tbLogBPM.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na tbLogBPM!


# Coletar dados da área dos batalhões da polícia militar

In [395]:
#Criar Data Frame com os dados dos BPMs
tbLogAreaBPM = pd.DataFrame(pd.read_csv("areaBPMv1.csv"))

#Criando coluna com a data da carga
tbLogAreaBPM['dtCarga'] = dtCarga

# Construindo a tabela de log de registros da área dos bpms (tblogareabpm), no ods

In [396]:
#Definir a query de criação da tabela
qry_tbLogAreaBPM = '''
    
    CREATE TABLE IF NOT EXISTS tbLogAreaBPM (
    
    codBPM INTEGER,
    areaBPM NUMERIC (7,2),
    dtCarga timestamp without time zone

    )
'''

#executar a criação da tbLogAreaBPM
SQL_ODS.execute(qry_tbLogAreaBPM)

#definição da qry de criação de index
qry_idx_tbLogAreaBPM_codBPM = "CREATE INDEX IF NOT EXISTS idx_tbLogAreaBPM_codBPM ON tbLogAreaBPM (codBPM)"

#executar a criação do index
SQL_ODS.execute(qry_idx_tbLogAreaBPM_codBPM)
connection.commit()

# Inserindo dados da área dos bpms na tabela tblogareabpm

In [397]:
#Inserindo registros na tabela tbLogAreaBPM
print("Iniciando a inserção de dados na tabela tbLogAreaBPM.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogAreaBPM (codBPM,areaBPM,dtCarga) VALUES (%s,%s,%s)''',tbLogAreaBPM.values.tolist())

#confirmar a transação
connection.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogAreaBPM), "registros inseridos na tbLogAreaBPM")

Iniciando a inserção de dados na tabela tbLogAreaBPM.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na tbLogAreaBPM


# Criando o dataframe com o select de criação da dimensão dp (dbpm), no dw

In [398]:
#Definir a query de criação da dimensão dBPM
qry_dBPM = '''
    SELECT
        codBPM,
        nmBPM,
        enderecoBPM,
        areaBPM
    FROM 
    (
        SELECT 
            a.codBPM,
            a.nmBPM,
            a.enderecoBPM,
            b.areaBPM,
            max(a.dtCarga)
        FROM tbLogBPM a
        JOIN tbLogAreaBPM b
        ON a.codBPM = b.codBPM
        WHERE a.dtCarga = (
                            SELECT MAX(x.dtCarga)
                            FROM tbLogBPM x
                       )
        GROUP BY
            a.codBPM,
            a.nmBPM,
            a.enderecoBPM,
            b.areaBPM
    ) a
'''

dBPM = pd.read_sql(qry_dBPM,connection)

# Coleta de dados dos registros de ocorrencias

In [399]:
#Dataframe com os dados dos registros das ocorrencias
tbLogOcorrencias = pd.DataFrame(pd.read_csv('OcorrenciaV2.csv'))

#Criando coluna com dtCarga
dtCarga = datetime.today().strftime('%m/%d/%y %H:%M')
tbLogOcorrencias['dtCarga'] = dtCarga

# Criando tabela tbOcorrencias em SQL

In [400]:
#Criando query
qry_create_tbOcorrencias = '''
CREATE TABLE IF NOT EXISTS tbLogOcorrencias (
    
    codDP INTEGER,
    codBPM INTEGER,
    ano INTEGER,
    mes INTEGER,
    mes_ano CHAR(7),
    regiao INTEGER,
    codMunicIBGE INTEGER,
    ocorrencias VARCHAR(100),
    qtde INTEGER,
    dtCarga timestamp without time zone
    )
'''
#executar a criação da Ocorrencias
SQL_ODS.execute(qry_create_tbOcorrencias)

#definição da qry de criação de index
qry_idx_tbOcorrencias = "CREATE INDEX IF NOT EXISTS idx_tbLogOcorrencias ON tbLogOcorrencias (codDP,codBPM,codMunicIBGE)"

#executar a criação do index
SQL_ODS.execute(qry_idx_tbOcorrencias)
connection.commit()

In [401]:
#inserindo dados na tbLogOcorrencias
print("Iniciando a carga de dados na tbLogOcorrencias")
print("-----------------------------------------------")
SQL_ODS.executemany('''

    INSERT INTO tbLogOcorrencias (codDP,codBPM,ano,mes,mes_ano,regiao,codMunicIBGE,ocorrencias,qtde,dtCarga)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)    
    ''', 
    tbLogOcorrencias.values.tolist()

)

#confirmar a transação
connection.commit()

print("Fim da carga de dados da tbLogOcorrencias!",len(tbLogOcorrencias),"Registros inseridos!")

Iniciando a carga de dados na tbLogOcorrencias
-----------------------------------------------
Fim da carga de dados da tbLogOcorrencias! 124007 Registros inseridos!


In [402]:
tbLogOcorrencias

,COD_DP,COD_BPM,ano,mes,mes_ano,Regiao,COD_Munic_IBGE,Ocorrencia,Soma de Qtde,dtCarga
0,1,5,2018,1,2018m01,1,3304557,ameaca,7,02/01/24 18:03
1,1,5,2018,1,2018m01,1,3304557,apreensao_drogas,3,02/01/24 18:03
2,1,5,2018,1,2018m01,1,3304557,estelionato,81,02/01/24 18:03
3,1,5,2018,1,2018m01,1,3304557,estupro,1,02/01/24 18:03
4,1,5,2018,1,2018m01,1,3304557,extorsao,3,02/01/24 18:03
...,...,...,...,...,...,...,...,...,...,...
124002,168,33,2018,12,2018m12,3,3304409,roubo_rua,1,02/01/24 18:03
124003,168,33,2018,12,2018m12,3,3304409,roubo_transeunte,1,02/01/24 18:03
124004,168,33,2018,12,2018m12,3,3304409,tentat_hom,2,02/01/24 18:03
124005,168,34,2018,12,2018m12,3,3304409,tentat_homicidio,2222222,02/01/24 18:03


# Criando DataFrame com o resultado da query de ocorrencias

In [403]:
qry_ocorrencias = '''
SELECT 
	a.coddp,
	a.codbpm,
	a.ano,
	a.mes,
	a.codmunicibge,
	a.ocorrencias,
	a.qtde,
    a.dtcarga
FROM tblogocorrencias a 
WHERE a.dtcarga = (SELECT max(dtcarga)
				  FROM tblogocorrencias)
'''
dfocorrencias = pd.read_sql(qry_ocorrencias, connection)


In [404]:
dfocorrencias

,coddp,codbpm,ano,mes,codmunicibge,ocorrencias,qtde,dtcarga
0,1,5,2018,1,3304557,ameaca,7,2024-02-01 18:03:00
1,1,5,2018,1,3304557,apreensao_drogas,3,2024-02-01 18:03:00
2,1,5,2018,1,3304557,estelionato,81,2024-02-01 18:03:00
3,1,5,2018,1,3304557,estupro,1,2024-02-01 18:03:00
4,1,5,2018,1,3304557,extorsao,3,2024-02-01 18:03:00
...,...,...,...,...,...,...,...,...
124002,168,33,2018,12,3304409,roubo_rua,1,2024-02-01 18:03:00
124003,168,33,2018,12,3304409,roubo_transeunte,1,2024-02-01 18:03:00
124004,168,33,2018,12,3304409,tentat_hom,2,2024-02-01 18:03:00
124005,168,34,2018,12,3304409,tentat_homicidio,2222222,2024-02-01 18:03:00


# Fechando conexão com o ODS

In [405]:
connection.commit()
connection.close()

# Conetando no DW e informando ao python que usarei linguagem SQL

In [406]:
dadosIBGE = dadosIBGE[['nmMunic','codMunic']]

host = "localhost"  
port = "5432"       
database = "pascoaDW"  
user = "postgres"  
password = "*****" 

# Conectando ao PostgreSQL
try:
    connection = sql.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Conexão ao PostgreSQL bem-sucedida!")
    
except sql.Error as e:
    print("Erro ao conectar ao PostgreSQL:", e)


Conexão ao PostgreSQL bem-sucedida!


In [407]:
#Definição de manipulação de dados em SQL
SQL_DW = connection.cursor()

In [408]:
#Definir a query de criação da tabela
qry_dmunicipio = '''
    
    CREATE TABLE IF NOT EXISTS dmunicipio (
    
    codMunic SERIAL PRIMARY KEY,
    nmMunic VARCHAR (100)
    

    )
'''

#executar a criação da tbLogBPM
SQL_DW.execute(qry_dmunicipio)

#executar a criação do index codBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dmunicipio_codMunic ON dmunicipio (codMunic)")
connection.commit()

# Deletando dados antigos e reiniciando ao autoincremental da chave

In [409]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dmunicipio")

#Reiniciando o autoincramental da chave
SQL_DW.execute("ALTER SEQUENCE dmunicipio_codMunic_seq RESTART WITH 1;")

#Confirmar a transação
connection.commit()

# Inserindo dados na dimensão dmunicipio (ddp), no dw

In [410]:
#Inserindo registros na tabela dmunicipio
print("Iniciando a inserção de dados na tabela dmunicipio.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dmunicipio (nmMunic, codMunic) VALUES (%s, %s)''',dadosIBGE.values.tolist())

#confirmar a transação
connection.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dadosIBGE), "registros inseridos na dmunicipio!")

Iniciando a inserção de dados na tabela dmunicipio.
-------------------------------------------------
Carga finalizada! 92 registros inseridos na dmunicipio!


# Deletando dados antigos e reiniciando ao autoincremental da chave

In [411]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dDP")

#Reiniciando o autoincramental da chave
SQL_DW.execute("ALTER SEQUENCE dDP_idDP_seq RESTART WITH 1;")

#Confirmar a transação
connection.commit()

# Inserindo dados na dimensão dp (ddp), no dw

In [412]:
#Inserindo registros na tabela tbLogRespDP
print("Iniciando a inserção de dados na tabela dDP.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dDP (cod_DP, nm_dp, endereco, responsavel) VALUES (%s, %s, %s, %s)''',dfdDP.values.tolist())

#confirmar a transação
connection.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dfdDP), "registros inseridos na dDP!")

Iniciando a inserção de dados na tabela dDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na dDP!


# Validando carga dos dados da dimensão dp (ddp)

In [413]:
pd.read_sql("SELECT * FROM dDP",connection)

,iddp,cod_dp,nm_dp,endereco,responsavel
0,1,121,121ª DP - Casimiro de Abreu,"Rua Geni da Cruz Leite, s/n - Centro, Casimiro...",Delegado de Polícia Ivailson Moreira Sardinha
1,2,157,157ª DP - Trajano de Moraes,"Alameda Rui Barbosa, s/n - Centro, Trajano de ...",Delegado de Polícia Franquis Dias Nepomuceno
2,3,106,106ª DP - Itaipava,"Estrada União e Indústria, 8764 - Nogueira, Pe...",Delegado de Polícia Nei José Ramos Loureiro
3,4,134,134ª DP - Campos,"Rua Barão de Miracema, 231 - Centro, Campos do...",Delegado de Polícia Ronaldo Andrade Cavalcante
4,5,75,075ª DP - Rio do Ouro,"Estrada Velha de Marica, s/n - Rio do Ouro, Sã...",Delegado de Polícia Cláudio Vieira de Campos
...,...,...,...,...,...
132,133,140,140ª DP - Natividade,"Avenida Governador Roberto Silveira, s/n - Nos...",Delegado de Polícia Uillian Rodrigues da Costa
133,134,50,050ª DP - Itaguaí,"Rua General Bocaiúva, 410 - Centro, Itaguaí - ...",Delegado de Polícia Marco Aurelio Castro
134,135,53,053ª DP - Mesquita,"Avenida Presidente Costa e Silva, 1289 - Centr...",Delegada de Polícia Juliana Emerique de Amorim...
135,136,13,013ª DP - Ipanema,"Avenida Nossa Senhora de Copacabana, 1260 - Co...",Delegado de Polícia Felipe Santoro da Silva


# Construindo tabela de dimensão período (dperiodo) no DW

In [414]:
#Definir a query de criação da tabela
qry_dperiodo = '''
    
    CREATE TABLE IF NOT EXISTS dperiodo (
    
    idperiodo SERIAL PRIMARY KEY,
    data timestamp without time zone,
    mes INTEGER,
    ano INTEGER,
    trimestre INTEGER,
    semestre INTEGER
    )
'''

#executar a criação da dperiodo
SQL_DW.execute(qry_dperiodo)

#executar a criação do index idperiodo
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dperiodo_idperiodo ON dperiodo (idperiodo)")

#executar a criação do index codBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_codBPM ON dperiodo (mes, ano, trimestre, semestre)")
connection.commit()

# Construindo a tabela de dimensão bpm (dbpm), no dw

In [415]:
#Definir a query de criação da tabela
qry_dBPM = '''
    
    CREATE TABLE IF NOT EXISTS dBPM (
    
    idBPM SERIAL PRIMARY KEY,
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    areaBPM NUMERIC (7,2)

    )
'''

#executar a criação da tbLogBPM
SQL_DW.execute(qry_dBPM)

#executar a criação do index idBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_idBPM ON dBPM (idBPM)")

#executar a criação do index codBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_codBPM ON dBPM (codBPM)")
connection.commit()

# Deletando dados antigos e reiniciando ao autoincremental da chave

In [416]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dBPM")

#Reiniciando o autoincramental da chave
SQL_DW.execute("ALTER SEQUENCE dBPM_idBPM_seq RESTART WITH 1;")

#Confirmar a transação
connection.commit()

#-- Desativar restrições
SQL_DW.execute("ALTER TABLE focorrencias DROP CONSTRAINT focorrencias_iddp_fkey;")

#-- Realizar a exclusão
SQL_DW.execute("DELETE FROM dBPM")

#-- Ativar restrições
SQL_DW.execute("ALTER TABLE focorrencias ADD CONSTRAINT focorrencias_iddp_fkey FOREIGN KEY (iddp) REFERENCES dDP (iddp);")
SQL_DW.execute("ALTER SEQUENCE dBPM_idBPM_seq RESTART WITH 1;")
connection.commit()

# Inserindo dados na dimensão dp (ddp), no dw

In [417]:
#Inserindo registros na tabela dBPM
print("Iniciando a inserção de dados na tabela dBPM.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dBPM (codBPM, nmBPM, enderecoBPM, areaBPM) VALUES (%s, %s, %s, %s)''',dBPM.values.tolist())

#confirmar a transação
connection.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dBPM), "registros inseridos na dBPM!")

Iniciando a inserção de dados na tabela dBPM.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na dBPM!


# Deletando dados antigos e reiniciando ao autoincremental da chave

In [418]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dperiodo")

#Reiniciando o autoincramental da chave
SQL_DW.execute("ALTER SEQUENCE dperiodo_idperiodo_seq RESTART WITH 1;")

#Confirmar a transação
connection.commit()

# Inserindo dados na dimensão dp (ddp), no dw

In [419]:
#Inserindo registros na tabela data
print("Iniciando a inserção de dados na tabela dperiodo.")
print("-------------------------------------------------")

#Criando query 
query_dperiodo = '''
WITH RECURSIVE data(d) AS (
    VALUES ('2018-01-01'::date)
    UNION ALL
    SELECT (d + INTERVAL '1 month')::date
    FROM data
    WHERE d < (date('now')::date)
)
SELECT 
    TO_CHAR(d, 'dd-mm-YYYY') AS Data,
    CAST(TO_CHAR(d, 'MM') AS INTEGER) AS Mes,
	CAST(TO_CHAR(d, 'yyyy') AS INTEGER) AS Ano,
	CASE WHEN CAST(TO_CHAR(d, 'MM') AS INTEGER) in (1, 2, 3) THEN 1
	WHEN CAST(TO_CHAR(d, 'MM') AS INTEGER) in (4, 5, 6) THEN 2
	WHEN CAST(TO_CHAR(d, 'MM') AS INTEGER) in (7, 8, 9) THEN 3
	else 3
	end as Trimestre,
	CASE WHEN CAST(TO_CHAR(d, 'MM')AS INTEGER) IN (1, 2, 3, 4, 5, 6) THEN 1
	else 2
	end as Semestre
	
FROM data;
'''

#Executando a query e carregando na tabela dperiodo
dperiodo = pd.read_sql(query_dperiodo, connection)

dperiodo['data'] = pd.to_datetime(dperiodo['data'], format='%m-%d-%Y').dt.strftime('%Y-%m-%d')
SQL_DW.executemany('''INSERT INTO dperiodo (data, mes, ano, trimestre, semestre) VALUES (%s, %s, %s, %s, %s)''',dperiodo.values.tolist())

#confirmar a transação
connection.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dperiodo), "registros inseridos na dBPM!")

Iniciando a inserção de dados na tabela dperiodo.
-------------------------------------------------
Carga finalizada! 74 registros inseridos na dBPM!


# Criando a tabela fato ocorrencias no DW

In [420]:
#definindo a qry da criação da fOcorrencias

qry_tbfocorrencias = '''
create table if not exists fOcorrencias(
	idDP INTEGER REFERENCES DDP(idDP) ON UPDATE CASCADE ON DELETE CASCADE,
    idbpm INTEGER REFERENCES dbpm(idbpm) ON UPDATE NO ACTION ON DELETE NO ACTION,
    idperiodo INTEGER REFERENCES dperiodo(idperiodo) ON UPDATE NO ACTION ON DELETE NO ACTION,
    codMunic INTEGER REFERENCES dMunicipio(codMunic) ON UPDATE NO ACTION ON DELETE NO ACTION,
    regiao INTEGER,
    ocorrencias VARCHAR(100),
    qtde INTEGER
)
'''
SQL_DW.execute(qry_tbfocorrencias)

#Criando index 
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_fOcorrencias ON fOcorrencias(idDP,idBPM,idPeriodo,codMunic)")
               


# Inserindo dados na fato ocorrências (focorrencias) - carga incremental

In [421]:
#criar um tabela temporária
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")
dfocorrencias.to_sql('tempOcorrencias', engine, if_exists="replace", index=False)

In [422]:
connection.commit()

In [423]:
dfocorrencias

,coddp,codbpm,ano,mes,codmunicibge,ocorrencias,qtde,dtcarga
0,1,5,2018,1,3304557,ameaca,7,2024-02-01 18:03:00
1,1,5,2018,1,3304557,apreensao_drogas,3,2024-02-01 18:03:00
2,1,5,2018,1,3304557,estelionato,81,2024-02-01 18:03:00
3,1,5,2018,1,3304557,estupro,1,2024-02-01 18:03:00
4,1,5,2018,1,3304557,extorsao,3,2024-02-01 18:03:00
...,...,...,...,...,...,...,...,...
124002,168,33,2018,12,3304409,roubo_rua,1,2024-02-01 18:03:00
124003,168,33,2018,12,3304409,roubo_transeunte,1,2024-02-01 18:03:00
124004,168,33,2018,12,3304409,tentat_hom,2,2024-02-01 18:03:00
124005,168,34,2018,12,3304409,tentat_homicidio,2222222,2024-02-01 18:03:00


In [424]:
#definindo a query de carga incremental
qry_carga_incremental_focorrencias = '''

    SELECT
        b.idDP,
        c.idBPM,
        d.idPeriodo,
        e.codMunic,
        a.ocorrencias,
        a.qtde
    FROM public."tempOcorrencias" a
    JOIN dDP b
    ON a.codDP = b.cod_dp
    JOIN dBPM c
    ON a.codBPM = c.codBPM
    JOIN dPeriodo d
    ON (a.ano = d.ano) AND (a.mes = d.mes)
    JOIN dMunicipio e
    ON a.codMunicIBGE = e.codMunic
    LEFT JOIN fOcorrencias g
    ON b.idDP = g.idDP
    AND c.idBPM = g.idBPM
    AND d.idPeriodo = g.idPeriodo
    AND e.codMunic = g.codMunic
    WHERE g.idDP is null
    AND g.idBPM is null
    AND g.idPeriodo is null
    AND g.codMunic is null

'''

#criando um DF Incremental
carga_fOcorrencias = pd.read_sql(qry_carga_incremental_focorrencias,connection)

carga_fOcorrencias

,iddp,idbpm,idperiodo,codmunic,ocorrencias,qtde
0,48,4,1,3304557,ameaca,7
1,48,4,1,3304557,apreensao_drogas,3
2,48,4,1,3304557,estelionato,81
3,48,4,1,3304557,estupro,1
4,48,4,1,3304557,extorsao,3
...,...,...,...,...,...,...
117914,108,11,12,3304409,roubo_rua,1
117915,108,11,12,3304409,roubo_transeunte,1
117916,108,11,12,3304409,tentat_hom,2
117917,108,22,12,3304409,tentat_homicidio,2222222


# Carregando dados na tabela fOcorrencias

In [425]:
print('Carregando dados na tabela fOcorrencias.')
print('------------------------------------------')

SQL_DW.executemany('''
insert into fOcorrencias (idDP,idBPM,idPeriodo,codMunic,ocorrencias,qtde)
    VALUES(%s,%s,%s,%s,%s,%s)''', carga_fOcorrencias.values.tolist()
)

#confirmar a transação 
connection.commit()

print("Fim da carga incremental de dados na fOcorrencias!",len(carga_fOcorrencias),"registros inseridos!")

Carregando dados na tabela fOcorrencias.
------------------------------------------
Fim da carga incremental de dados na fOcorrencias! 117919 registros inseridos!


In [426]:
# Deletando a tabela temporaria
SQL_DW.execute('drop table public."tempOcorrencias"')

# Fechando a conexão com o DW

In [427]:
connection.commit()
connection.close()

In [428]:
print('')